# Data Preparation - Viral Social Media Trends Analysis

In [ ]:
# Imports
import json
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Imports loaded")

: 

## Configurations

In [ ]:
# Paths
RAW_DATA_PATH = "data/raw/Cleaned_Viral_Social_Media_Trends.csv"
PROCESSED_DIR = Path("data/processed")
ARTIFACTS_DIR = Path("artifacts")
FIGURES_DIR = Path("artifacts/figures")

# Create directories
for dir_path in [PROCESSED_DIR, ARTIFACTS_DIR, FIGURES_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)
    # Clear all files in directory
    for file in dir_path.glob('*'):
        if file.is_file():
            file.unlink()

# Constants
# TODO : Implement needed constraints

print("✅ Configuration loaded")

## Helper Functions

In [ ]:
def calculate_iqr_bounds(series, multiplier=1.5):
    """Calculate IQR-based outlier bounds"""
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1 - multiplier * iqr, q3 + multiplier * iqr

# TODO : Implement other Functions here 

def build_preprocessor(nominal_cols, ordinal_cols, ordinal_categories, numeric_cols):
    """Build preprocessing pipeline - Steps 6 & 7"""
    transformers = [
        ('num', Pipeline([
            ('scaler', StandardScaler())
        ]), numeric_cols),
        ('nom', Pipeline([
            ('onehot', OneHotEncoder(handle_unknown='ignore', min_frequency=0.01, sparse_output=False))
        ]), nominal_cols)
    ]
    
    if ordinal_cols:
        transformers.append(('ord', Pipeline([
            ('ordinal', OrdinalEncoder(categories=ordinal_categories, handle_unknown='use_encoded_value', unknown_value=-1))
        ]), ordinal_cols))
    
    return ColumnTransformer(transformers)

def extract_feature_names(fitted_ct):
    """Extract feature names from fitted ColumnTransformer"""
    names = []
    for name, transformer, columns in fitted_ct.transformers_:
        if name == 'num':
            names.extend(columns)
        elif name == 'nom':
            names.extend(transformer.named_steps['onehot'].get_feature_names_out(columns))
        elif name == 'ord':
            names.extend(columns)
    return names

print("✅ Helper functions defined")


## Load Raw Data

In [ ]:
df_raw = pd.read_csv(RAW_DATA_PATH)
print(f"✅ Loaded {len(df_raw):,} rows, {len(df_raw.columns)} columns")

# Drop unnecessary text columns immediately
df_raw = df_raw.drop(columns=['Post_ID'], errors='ignore')
print(f"✅ Dropped host_name and name columns")

print(f"\nColumns: {list(df_raw.columns)}")
df_raw.head(3)


In [ ]:
# Missing value statistics
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Pct': missing_pct.values
}).sort_values('Missing_Count', ascending=False)

print("=" * 80)
print("MISSING VALUE ANALYSIS")
print("=" * 80)
print(missing_df[missing_df['Missing_Count'] > 0].to_string(index=False))

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Heatmap
sns.heatmap(df_raw.isnull(), cbar=False, yticklabels=False, cmap='viridis', ax=ax1)
ax1.set_title('Missing Value Pattern (Yellow = Missing)', fontsize=14, fontweight='bold')

# Bar chart
missing_nonzero = missing_df[missing_df['Missing_Count'] > 0]
if len(missing_nonzero) > 0:
    ax2.barh(missing_nonzero['Column'], missing_nonzero['Missing_Pct'], color='coral', edgecolor='black')
    ax2.set_xlabel('Missing %', fontsize=12)
    ax2.set_title('Missing Value Percentage by Column', fontsize=14, fontweight='bold')
    ax2.invert_yaxis()
    for i, (col, pct) in enumerate(zip(missing_nonzero['Column'], missing_nonzero['Missing_Pct'])):
        ax2.text(pct + 0.5, i, f'{pct:.1f}%', va='center', fontsize=10)
else:
    ax2.text(0.5, 0.5, 'No Missing Values', ha='center', va='center', fontsize=16, transform=ax2.transAxes)
    ax2.axis('off')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'missing_values_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Missing value analysis saved to {FIGURES_DIR / 'missing_values_analysis.png'}")
print(f"\n📝 Strategy: Impute after split (Step 3) using TRAIN statistics only")


## Exploratory Data Analysis

In [ ]:
print("Data Summary:")
print(df_raw.describe())
print(f"\n✅ EDA complete - Ready for STEP 2: Split")

In [ ]:
df_raw.describe()

In [ ]:
df_raw.describe(include='object')
for col in df_raw.select_dtypes(include='object').columns:
    print(f"\n{col}: {df_raw[col].nunique()} unique values")
    print(df_raw[col].unique())

In [ ]:
# Plot correlation matrix
plt.figure(figsize=(12, 8))
numeric_cols = df_raw.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = df_raw[numeric_cols].corr()

# Create heatmap
sns.heatmap(
            correlation_matrix, 
            annot=True,
            cmap='coolwarm',
            center=0,
            fmt='.2f',
            square=True,
            linewidths=0.5
            )

plt.title('Correlation Matrix of Numeric Features', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Correlation analysis saved to {FIGURES_DIR / 'correlation_matrix.png'}")


## Data Visualization

In [ ]:
# Histograms for distribution
for i in df_raw.select_dtypes(include="number").columns:
    sns.histplot(data=df_raw, x=i, kde=True, color="red")
    plt.title(f'Distribution of {i}')
    plt.savefig(FIGURES_DIR / f'Histogram-distribution of {i}.png', dpi=150, bbox_inches='tight')
    plt.tight_layout
    plt.show()

In [ ]:
count = df_raw['Platform'].value_counts()
colors = ["#FF0000", "#69C9D0", "#E1306C", "#1DA1F2"]
plt.pie(count, labels=count.index, colors=colors ,autopct='%1.1f%%')
plt.title('Platform Distribution', fontsize=15 , weight='bold')
plt.savefig(FIGURES_DIR / f'PieChart.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
region_distribution = df_raw['Region'].value_counts()
colors = [ "#001f54" ,"#d62828","#3a86ff","#f4d35e","#264653","#ffe066","#ff99ac","#cc5803","#001f54" ,"#d62828" ]
plt.pie(region_distribution, labels=region_distribution.index, autopct='%1.1f%%', colors=colors)
plt.title('Region Distribution', fontsize=15 , weight='bold')
plt.savefig(FIGURES_DIR / f'Region Piechart.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.barplot(x='Platform', y='Views',data=df_raw, palette='viridis',ci=False)
plt.title('Total Views by Platform', fontsize=15 , weight='bold')
plt.xlabel('Platform', weight='bold')
plt.ylabel('Total Views', weight='bold')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.savefig(FIGURES_DIR / f'Barchart-Platform.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
content_views = df_raw.groupby('Content_Type')['Views'].sum().reset_index()
sns.barplot(x='Content_Type', y='Views', data=content_views, palette='Set2')
plt.title('Total Views by Content Type', fontsize=15 , weight='bold')
plt.xlabel('Content Type', weight='bold')
plt.ylabel('Total Views', weight='bold')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.savefig(FIGURES_DIR / f'Barchart-ContentType.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
views_per_country = df_raw.groupby('Region')['Views'].sum()
sns.barplot(x=views_per_country.index, y=views_per_country.values , palette='Reds')
plt.title('Total Views per Country', fontsize=15 , weight='bold')
plt.xlabel('Country', weight='bold')
plt.ylabel('Total Views', weight='bold')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.savefig(FIGURES_DIR / f'Barchart-Country.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
most5_used_hastags = df_raw['Hashtag'].value_counts().sort_values(ascending=False).nlargest(5)
sns.barplot(x=most5_used_hastags.index, y=most5_used_hastags.values, palette='Greens')
plt.title('Top 5 Most Used Hashtags', fontsize=15 , weight='bold')
plt.xlabel('Hashtag', weight='bold')
plt.ylabel('Frequency', weight='bold')
plt.savefig(FIGURES_DIR / f'Barchart-Hashtags.png', dpi=150, bbox_inches='tight')
plt.show()

## Split Train/Test 

#### Define feature columns

In [ ]:
NOMINAL_COLS = ['Platform', 'Hashtag', 'Region', 'Content_Type']
ORDINAL_COLS = ['Engagement_level']
ORDINAL_CATEGORIES = [['High', 'Medium', 'Low']]
NUMERIC_COLS = ['Views', 'Likes', 'Shares', 'Comments']


feature_cols = NOMINAL_COLS + ORDINAL_COLS +  NUMERIC_COLS
targets = NUMERIC_COLS
target_cal = ORDINAL_COLS

X = df_raw[feature_cols].copy()
y_reg = df_raw[targets].copy()
y_cal = df_raw[target_cal].copy()

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

X_train_cal, X_test_cal, y_train_cal, y_test_cal = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

print(f"✅ Step 2 complete:")
print(f"For predicting continous values:")
print(f"   TRAIN: {len(X_train_reg):,} samples")
print(f"   TEST:  {len(X_test_reg):,} samples")
print(f"\n⚠️ NO FITTING HAS OCCURRED YET - ALL STATISTICS WILL USE TRAIN ONLY")

print(f"For classificaion:")
print(f"   TRAIN: {len(X_train_cal):,} samples")
print(f"   TEST:  {len(X_test_cal):,} samples")
print(f"\n⚠️ NO FITTING HAS OCCURRED YET - ALL STATISTICS WILL USE TRAIN ONLY")

# TODO : Continue other Steps after this Split and do other